# make a new function to make a 2D slice based on 1D Earth model (or 2D or 3D)

In [ ]:
# Nobuaki Fuji Oct 2025

# breaking news! confirmed by Lorette and Stéphanie, the CMB is ellipsoidal! 
# when working with geodetic earth models, r_PREM should be corrected as a function of latitude!

using Pkg


cd(@__DIR__)
Pkg.activate("../..")
ParamFile = "../test/testparam.csv"
include("../src/batchRevise.jl") 

myInclude("../src/DSM1D.jl")
using .DSM1D

  Activating 

In [ ]:
using Geodesy, Interpolations, StaticArrays, GMT, LinearAlgebra

In [ ]:
DSM1D.my1DDSMmodel.averagedPlanetRadiusInKilometer

In [ ]:
# Example 1-D model (toy). Replace with your actual model arrays.
# r_model in meters from Earth center (0..R_ref), assume sorted ascending
R_ref = DSM1D.my1DDSMmodel.averagedPlanetRadiusInKilometer*1.e3                   # model reference radius (m)
r_model = collect(0.0:1000.0:R_ref)  # example radius grid
v_model = 5000 .- 0.5 .* (R_ref .- r_model) ./ 1000  # dummy profile

In [ ]:
# create an interpolator in radius space (ensure interpolation domain covers used r)
itp = LinearInterpolation(r_model, v_model; extrapolation_bc=Flat())

In [ ]:
import Base: +,-,/
struct GeoPoint
    lat::Float64 # in degree
    lon::Float64 # in degree
    alt::Float64 # in metre
    ecef::SVector{3,Float64}
    radius::Float64 # in metre
    #effectiveRadius::Float64 # in metre (useful to get values from 1D averaged model)
end

function GeoPoint(lat::Float64, lon::Float64; alt=0.0, ell=wgs84)
    lla = LLA(lat,lon, alt) # be careful LLA uses degrees by default!!
    ecef_coords = ECEF(lla,ell)
    radius = norm([ecef_coords.x,ecef_coords.y,ecef_coords.z])
    GeoPoint(lat, lon, alt, SVector(ecef_coords.x, ecef_coords.y, ecef_coords.z),radius)
end


function GeoPoint(ecef::SVector{3,Float64}; ell=wgs84)
    lla = LLA(ECEF(ecef...),ell)
    radius = norm(ecef)
    GeoPoint(lla.lat,lla.lon,lla.alt,ecef,radius)
end

function +(a::GeoPoint,b::GeoPoint; ell=wgs84)
    ecef=a.ecef + b.ecef
    lla = LLA(ECEF(ecef...),ell)
    radius = norm(ecef)
    GeoPoint(lla.lat,lla.lon,lla.alt,ecef,radius)
end

function -(a::GeoPoint,b::GeoPoint; ell=wgs84)
    ecef=a.ecef - b.ecef
    lla = LLA(ECEF(ecef...),ell)
    radius = norm(ecef)
    GeoPoint(lla.lat,lla.lon,lla.alt,ecef,radius)
end

function /(a::GeoPoint,c::Real; ell=wgs84)
    ecef=a.ecef / c
    lla = LLA(ECEF(ecef...),ell)
    radius = norm(ecef)
    GeoPoint(lla.lat,lla.lon,lla.alt,ecef,radius)
end


function effectiveRadius(a::GeoPoint,r0::Float64; ell=wgs84)
    radiusPlanetHere = GeoPoint(a.lat,a.lon).radius 
    ratio = r0/radiusPlanetHere
    return a.radius*ratio
end

function topographyFromGeoPoint(p::GeoPoint; dlon=0.01, dlat=0.01, precision="@earth_relief_01m")
    region = [p.lon - dlon, p.lon + dlon, p.lat - dlat, p.lat + dlat]
    topo = GMT.grdcut(precision, region=region)
    return GMT.grd2xyz(topo).z[1]  # or mean(topo.z) if you prefer
end


In [ ]:
p1 = GeoPoint(48.8566,2.3522) # Paris
p2 = GeoPoint(42.8,1.5) # Tarascon (à peu près)
#topographyFromGeoPoint(p1)
@show GeoPoint(p1.ecef)
@show p0 = (p1 + p2)/2.0


In [ ]:
# pre-definition of x-axis vector p1->p2 (normally this is not the one we want)
p2_1 = p2-p1
x_axis_tentative = (p2_1/p2_1.radius).ecef


In [ ]:
# definition of z-axis vector p0 -> p0_surface

z_axis = (p0/p0.radius).ecef

In [ ]:
# y-axis: complete right-handed system
y_axis = normalize(cross(z_axis, x_axis_tentative))

In [ ]:
x_axis = cross(y_axis, z_axis)  # now perfectly orthogonal

In [ ]:
#Rotation matrix
R = SMatrix{3,3,Float64}(
    x_axis[1], y_axis[1], z_axis[1],
    x_axis[2], y_axis[2], z_axis[2],
    x_axis[3], y_axis[3], z_axis[3]
)

In [ ]:
p_2D_to_ECEF(x_2D,z_2D,pOrigin::SVector{3,Float64},R::SMatrix{3,3,Float64}) = pOrigin+R*SVector(x_2D,0.e0,z_2D)

In [ ]:
p_ECEF_to_2D(p_3D::SVector{3,Float64},pOrigin::SVector{3,Float64},R::SMatrix{3,3,Float64}) = R' * (p_3D - pOrigin)

In [ ]:
p_ECEF_to_2D(p0.ecef, p1.ecef,R)

In [ ]:
p0_1m_above=p_2D_to_ECEF(0,1,p1.ecef,R)

In [ ]:
p_ECEF_to_2D(p0_1m_above, p1.ecef,R)

In [ ]:
Δx = 500 # in metre
Δz = 500

altMax = 5000 # in metre
altMin = -400.e3 # in metre

In [ ]:
# since p1 and p2 can be even out of the solid Earth or inside the Earth
# I define that p1 and p2 should be the left and right limits

In [ ]:
leftLimit = 0 # in metre so x is always non-negative
rightLimit = p2_1.radius


In [ ]:
@show Nx = Int64((rightLimit-leftLimit) ÷ Δx+1) 
@show Nz = Int64((altMax-altMin) ÷ Δz + 1 ) 

In [ ]:
DSM1D.my1DDSMmodel.C_Qκ

In [ ]:
myRadiiArray=[5330.0,7330.0]

In [ ]:
radii,params=DSM1D.compute1DseismicParamtersFromPolynomialCoefficientsWithGivenRadiiArray(DSM1D.my1DDSMmodel,myRadiiArray,"below")
params.Vph

In [ ]:
allGridsInGeoPoints=Array{GeoPoint,2}(undef,Nx,Nz)
effectiveRadii=Array{Float64,2}(undef,Nx,Nz)
for iXZ in CartesianIndices(allGridsInGeoPoints)
    ix, iz = Tuple(iXZ)
    x = leftLimit+(ix-1)*Δx
    z = altMin+(iz-1)*Δz 
    tmpGeoPoint=GeoPoint(p_2D_to_ECEF(x,z,p1.ecef,R))
    allGridsInGeoPoints[iXZ]=tmpGeoPoint
    effectiveRadii[iXZ]=effectiveRadius(tmpGeoPoint,DSM1D.my1DDSMmodel.averagedPlanetRadiusInKilometer*1.e3 )
end

In [ ]:
allGridsInGeoPoints[3,3]

In [ ]:
effectiveRadii[200,1]

In [ ]:
minimum(effectiveRadii)

In [ ]:
# now I need to use 